In [12]:
import h5py
import numpy as np
from sklearn.model_selection import train_test_split

dataset_path = r'C:\Users\CyberVortex\PFE_Decoding\dataset\ldpc_dataset.h5'

# Charger les données
with h5py.File(dataset_path, 'r') as hf:
    X = hf['code_word'][:]  # Signal reçu (shape: (40000, 200))
    y = hf['message'][:]     # Bits d'info (shape: (40000, 100))

In [13]:
X.shape

(40000, 200)

In [14]:
y.shape

(40000, 100)

In [15]:
# Normalisation globale (calculée sur l'ensemble d'entraînement)
X_mean = np.mean(X)
X_std = np.std(X)
X = (X - X_mean) / X_std

In [16]:
# Mélanger les données
indices = np.random.permutation(len(X))
X_shuffled = X[indices]
y_shuffled = y[indices]

# Séparation (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X_shuffled, y_shuffled, test_size=0.2, random_state=42)

In [17]:
# Redimensionner pour LSTM/Transformer (ajouter une dimension de canal)
X_train_3d = X_train[..., np.newaxis]  # Shape: (32000, 200, 1)
X_val_3d = X_val[..., np.newaxis]       # Shape: (8000, 200, 1)

In [20]:
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model

# Paramètres
input_shape = (200, 1)  # 200 symboles, 1 canal
k = 100                  # Nombre de bits d'information

# Architecture du modèle
inputs = Input(shape=input_shape)
x = LSTM(128, return_sequences=True)(inputs)
x = LSTM(64)(x)
outputs = Dense(k, activation='sigmoid')(x)  # k=100

model = Model(inputs, outputs)

In [21]:
import numpy as np

# Redimensionner X pour qu'il ait 3 dimensions : (échantillons, pas de temps, caractéristiques)
X = X.reshape((X.shape[0], X.shape[1], 1))  # Shape: (40000, 200, 1)

In [22]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # Pour une classification binaire bit à bit
    metrics=['accuracy']
)

# Entraînement
history = model.fit(
    X, y,
    batch_size=128,
    epochs=10,
    validation_split=0.2  # 80% train, 20% validation
)

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 93s 364ms/step - accuracy: 0.0067 - loss: 0.6932 - val_accuracy: 0.0024 - val_loss: 0.6932
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 88s 352ms/step - accuracy: 0.0056 - loss: 0.6932 - val_accuracy: 0.0031 - val_loss: 0.6932
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 89s 357ms/step - accuracy: 0.0150 - loss: 0.6932 - val_accuracy: 0.0000e+00 - val_loss: 0.6932
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 90s 362ms/step - accuracy: 0.0036 - loss: 0.6931 - val_accuracy: 0.0108 - val_loss: 0.6932
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 91s 363ms/step - accuracy: 0.0049 - loss: 0.6931 - val_accuracy: 0.0018 - val_loss: 0.6932
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 115s 462ms/step - accuracy: 0.0097 - loss: 0.6931 - val_accuracy: 0.0051 - val_loss: 0.6932
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 104s 415ms/step - accuracy: 0.0055 - loss: 0.6931 - val_accuracy: 0.0040 - val_loss: 0.6932
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 91s 363ms/step - accuracy: 0.0050 - l